In [1]:
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
import unicodedata
import re
import utils as utils
from calendar import monthrange

In [2]:
source_db = "trusted"
source_table = "f_ocupacion_hotelera"
db_name = "exploitation"
table_name = "f_ocupacion_barcelona"

In [3]:
spark = utils.create_context()

In [11]:
df_ocupacion_hotelera = utils.read_iceberg_table(spark,source_db,source_table)

In [5]:
@udf(IntegerType())
def dias_mes(year,month):

    days = monthrange(year,month)[1]

    return days


In [12]:
df_ocupacion_hotelera.filter(col("provincia").like("%BAR%")).show(20,False)

+----+---+--------------+---------------------------+---------+--------+--------------+-------------+-------------------------+---------------------+---------------+----------------+-------------------------+-------------------------+----------------+
|año |mes|puntoTuristico|lugarResidencia            |provincia|viajeros|pernoctaciones|estanciaMedia|establecimientosEstimados|habitacionesEstimadas|plazasEstimadas|gradoOcupaPlazas|gradoOcupaPlazasFinSemana|gradoOcupaPorHabitaciones|personalEmpleado|
+----+---+--------------+---------------------------+---------+--------+--------------+-------------+-------------------------+---------------------+---------------+----------------+-------------------------+-------------------------+----------------+
|2012|1  |BARCELONA     |RESIDENTES EN ESPANA       |BARCELONA|123303  |210821        |1.71         |NULL                     |NULL                 |NULL           |NULL            |NULL                     |NULL                     |NULL      

In [13]:
df_ocupacion_hotelera = (
    df_ocupacion_hotelera
    .filter(col("lugarResidencia") == "TOTAL")
    .filter(col("provincia") == "BARCELONA")
    .withColumn(
        "diasMes",
        dias_mes(col("año"),col("mes"))
    )
    .withColumn(
        "TasaOcupacion",
        when(
            (col("plazasEstimadas").isNotNull()) & (col("plazasEstimadas") > 0),
            (col("pernoctaciones") / (col("plazasEstimadas") * col("diasMes"))) * 100
        ).otherwise(None)
    )
)



In [14]:
df_ocupacion_hotelera.show()

+----+---+--------------+---------------+---------+--------+--------------+-------------+-------------------------+---------------------+---------------+----------------+-------------------------+-------------------------+----------------+-------+------------------+
| año|mes|puntoTuristico|lugarResidencia|provincia|viajeros|pernoctaciones|estanciaMedia|establecimientosEstimados|habitacionesEstimadas|plazasEstimadas|gradoOcupaPlazas|gradoOcupaPlazasFinSemana|gradoOcupaPorHabitaciones|personalEmpleado|diasMes|     TasaOcupacion|
+----+---+--------------+---------------+---------+--------+--------------+-------------+-------------------------+---------------------+---------------+----------------+-------------------------+-------------------------+----------------+-------+------------------+
|2012|  1|     BARCELONA|          TOTAL|BARCELONA|  377320|        848258|         2.25|                      491|                 NULL|          66531|            41.0|                    43.41|   

In [15]:
utils.overwrite_iceberg_table(spark,df_ocupacion_hotelera,db_name,table_name)